<a name="top"></a><img src="./source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)   

In [1]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/domenic/Documents/Spinal-bootcamp/source/load-spinal.sc"

## Assignement overlap
SpinalHDL will check that no signal assignement completly erases a previous one.   
The following code will throw the following error:

In [2]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  a := 66 //Erease the a := 42 :(
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:44:01
[Progress] at 0.000 : Elaborate components
[Progress] at 0.094 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 0.144 : Elaborate components
[Progress] at 0.146 : Checks and transforms


: 

A fix could be:

In [2]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  when(something){
    a := 66
  }
}
showRtl(new TopLevel)

cmd2.sc:4: not found: value something
  when(something){
       ^Compilation Failed

: 

But in the case you really want to override the previous assignement (Yes, it could make sense in some cases), you can do the following:

In [3]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  a.allowOverride
  a := 66
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:45:54
[Progress] at 112.497 : Elaborate components
[Progress] at 112.499 : Checks and transforms
[Progress] at 112.517 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 112.597
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  reg        [7:0]    a;
  function [7:0] zz_a(input dummy);
    begin
      zz_a = 8'h2a;
      zz_a = 8'h42;
    end
  endfunction
  wire [7:0] _zz_1;

  assign _zz_1 = zz_a(1'b0);
  always @ (*) a = _zz_1;

endmodule



defined class TopLevel

## Clock crossing violation
SpinalHDL will check that every register of your design only depends (through some combinatorial logic) on registers which use the same or a synchronous clock domain. 

The following code will throw error:

In [4]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))   //PlayDev.scala:834
  val regB = clkB(Reg(UInt(8 bits)))   //PlayDev.scala:835

  val tmp = regA + regA                //PlayDev.scala:838
  regB := tmp
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:47:24
[Progress] at 202.537 : Elaborate components
[Progress] at 202.547 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 202.569 : Elaborate components
[Progress] at 202.572 : Checks and transforms


: 

There are multiple possible fixes:
### crossClockDomain tag
The crossClockDomain tag can be used to say “It’s alright, don’t panic” to SpinalHDL

In [5]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits))).addTag(crossClockDomain)


  val tmp = regA + regA
  regB := tmp
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:48:05
[Progress] at 243.524 : Elaborate components
[Progress] at 243.527 : Checks and transforms
[Progress] at 243.531 : Generate Verilog
[Warning] 7 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 243.557
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  input               clkA_clk,
  input               clkA_reset,
  input               clkB_clk,
  input               clkB_reset
);
  reg        [7:0]    regA;
  reg        [7:0]    regB;
  wire       [7:0]    tmp;

  assign tmp = (regA + regA);
  always @ (posedge clkB_clk) begin
    regB <= tmp;
  end


endmodule



defined class TopLevel

### setSyncronousWith
You can also specify that two clock domains are syncronous together.

In [6]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")
  clkB.setSyncronousWith(clkA)

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits)))


  val tmp = regA + regA
  regB := tmp
}

showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:48:49
[Progress] at 287.822 : Elaborate components
[Progress] at 287.825 : Checks and transforms
[Progress] at 287.830 : Generate Verilog
[Warning] 7 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 287.847
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  input               clkA_clk,
  input               clkA_reset,
  input               clkB_clk,
  input               clkB_reset
);
  reg        [7:0]    regA;
  reg        [7:0]    regB;
  wire       [7:0]    tmp;

  assign tmp = (regA + regA);
  always @ (posedge clkB_clk) begin
    regB <= tmp;
  end


endmodule



defined class TopLevel

### BufferCC
Signal Bits or Gray-coded Bits can use BufferCC to cross different clockDomain

In [7]:
class syncRead2Write extends Component {
  val io = new Bundle{
    val pushClock, pushRst = in Bool()
    val readPtr = in UInt(8 bits)
  }
  val pushCC = new ClockingArea(ClockDomain(io.pushClock, io.pushRst)) {
    val pushPtrGray = RegNext(toGray(io.readPtr)) init(0)
  }
}
showRtl(new syncRead2Write)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:50:55
[Progress] at 413.655 : Elaborate components
[Progress] at 413.676 : Checks and transforms
[Progress] at 413.684 : Generate Verilog
[Warning] 4 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 413.715
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : syncRead2Write
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module syncRead2Write (
  input               io_pushClock,
  input               io_pushRst,
  input      [7:0]    io_readPtr
);
  wire       [7:0]    _zz_1;
  reg        [7:0]    pushCC_pushPtrGray;

  assign _zz_1 = (io_readPtr >>> 1'b1);
  always @ (posedge io_pushClock or posedge io_pushRst) begin
    if (io_pushRst) begin
      pushCC_pushPtrGray <= 8'h0;
    end else begin
      pushCC_pushPtrG

defined class syncRead2Write

## Combinational loop
SpinalHDL will check that there are no combinatorial loops in the design.

The following code will throw error:

In [8]:
class TopLevel extends Component {
  val a = UInt(8 bits) //PlayDev.scala line 831
  val b = UInt(8 bits) //PlayDev.scala line 832
  val c = UInt(8 bits)
  val d = UInt(8 bits)

  a := b
  b := c | d
  d := a
  c := 0
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:52:54
[Progress] at 532.491 : Elaborate components
[Progress] at 532.493 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 532.504 : Elaborate components
[Progress] at 532.506 : Checks and transforms


: 

A possible fix could be:

In [9]:
class TopLevel extends Component {
  val a = UInt(8 bits) //PlayDev.scala line 831
  val b = UInt(8 bits) //PlayDev.scala line 832
  val c = UInt(8 bits)
  val d = UInt(8 bits)

  a := b
  b := c | d
  d := 42
  c := 0
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:53:48
[Progress] at 586.391 : Elaborate components
[Progress] at 586.392 : Checks and transforms
[Progress] at 586.396 : Generate Verilog
[Warning] 4 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 586.430
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire       [7:0]    a;
  wire       [7:0]    b;
  wire       [7:0]    c;
  wire       [7:0]    d;

  assign a = b;
  assign b = (c | d);
  assign d = 8'h2a;
  assign c = 8'h0;

endmodule



defined class TopLevel

### False-positive
It should be said that SpinalHDL’s algorithm to detect combinatorial loops can be pessimistic, and it may give false positives. If it is giving a false positive, you can manualy disable loop checking on one signal of the loop like so:

In [10]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 0
  a(1) := a(0) //False positive because of this line
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:54:47
[Progress] at 645.029 : Elaborate components
[Progress] at 645.032 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 645.038 : Elaborate components
[Progress] at 645.040 : Checks and transforms


: 

could be fixed by :

In [11]:
class TopLevel extends Component {
  val a = UInt(8 bits).noCombLoopCheck
  a := 0
  a(1) := a(0)
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:54:58
[Progress] at 656.354 : Elaborate components
[Progress] at 656.354 : Checks and transforms
[Progress] at 656.357 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 656.391
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  reg        [7:0]    a;

  always @ (*) begin
    a = 8'h0;
    a[1] = a[0];
  end


endmodule



defined class TopLevel

It should also be said that assignments such as (a(1) := a(0)) can make some tools like Verilator unhappy. It may be better to use a Vec(Bool, 8) in this case

## Hierarchy violation

SpinalHDL will check that signals are never accessed outside of the current component’s scope.

The following signals can be read inside a component:

- All directionless signals defined in the current component

- All in/out/inout signals of the current component

- All in/out/inout signals of children components

In addition, the following signals can be assigned to inside a component:

- All directionless signals defined in the current component

- All out/inout signals of the current component

- All in/inout signals of children components

If a `HIERARCHY VIOLATION`error appears, it means that one of the above rules was violated.

The following code will throw error:

In [12]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
  val tmp = U"x42"
  io.a := tmp
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:57:41
[Progress] at 819.987 : Elaborate components
[Progress] at 819.991 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 819.994 : Elaborate components
[Progress] at 819.996 : Checks and transforms


: 

A fix could be :

In [13]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = out UInt(8 bits) // changed from in to out
  }
  val tmp = U"x42"
  io.a := tmp
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:58:06
[Progress] at 844.160 : Elaborate components
[Progress] at 844.163 : Checks and transforms
[Progress] at 844.166 : Generate Verilog
[Done] at 844.184
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  output     [7:0]    io_a
);
  wire       [7:0]    tmp;

  assign tmp = 8'h42;
  assign io_a = tmp;

endmodule



defined class TopLevel

## Io bundle
SpinalHDL will check that each io bundle contains only in/out/inout signals.

In [14]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = UInt(8 bits)
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:59:41
[Progress] at 939.086 : Elaborate components
[Progress] at 939.088 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 939.091 : Elaborate components
[Progress] at 939.092 : Checks and transforms


: 

A fix could be:

In [15]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 09:59:48
[Progress] at 946.852 : Elaborate components
[Progress] at 946.854 : Checks and transforms
[Progress] at 946.856 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 946.880
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  input      [7:0]    io_a
);


endmodule



defined class TopLevel

But if for meta hardware description reasons you realy want io.a to be directionless, you can do:

In [16]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = UInt(8 bits)
  }
  io.a.allowDirectionLessIo
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:00:04
[Progress] at 962.413 : Elaborate components
[Progress] at 962.415 : Checks and transforms
[Progress] at 962.417 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 962.437
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire       [7:0]    io_a;


endmodule



defined class TopLevel

## Latch detected
SpinalHDL will check that no combinatorial signal will infer a latch during synthesis. In other words, that no combinatorial signals are partialy assigned.

In [17]:
class TopLevel extends Component {
  val cond = in(Bool)
  val a = UInt(8 bits)

  when(cond){
    a := 42
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:00:27
[Progress] at 985.088 : Elaborate components
[Progress] at 985.092 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 985.101 : Elaborate components
[Progress] at 985.102 : Checks and transforms


: 

A fix could be:

In [18]:
class TopLevel extends Component {
  val cond = in(Bool)
  val a = UInt(8 bits)

  a := 0
  when(cond){
    a := 42
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:01:02
[Progress] at 1020.366 : Elaborate components
[Progress] at 1020.367 : Checks and transforms
[Progress] at 1020.371 : Generate Verilog
[Warning] 2 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1020.406
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  input               cond
);
  reg        [7:0]    a;

  always @ (*) begin
    a = 8'h0;
    if(cond)begin
      a = 8'h2a;
    end
  end


endmodule



defined class TopLevel

## No driver on
SpinalHDL will check that all combinatorial signals which have an impact on the design are assigned by something.

In [19]:
class TopLevel extends Component {
  val result = out(UInt(8 bits))
  val a = UInt(8 bits)
  result := a
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:01:53
[Progress] at 1071.628 : Elaborate components
[Progress] at 1071.629 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 1071.632 : Elaborate components
[Progress] at 1071.633 : Checks and transforms


: 

A fix could be:

In [20]:
class TopLevel extends Component {
  val result = out(UInt(8 bits))
  val a = UInt(8 bits)
  a := 42
  result := a
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:02:11
[Progress] at 1089.433 : Elaborate components
[Progress] at 1089.434 : Checks and transforms
[Progress] at 1089.436 : Generate Verilog
[Done] at 1089.458
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  output     [7:0]    result
);
  wire       [7:0]    a;

  assign a = 8'h2a;
  assign result = a;

endmodule



defined class TopLevel

## NullPointerException
NullPointerException is a Scala runtime reported error which can happen when a variable is accessed before it was initialised.

In [21]:
class TopLevel extends Component {
  a := 42
  val a = UInt(8 bits)
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:02:34
[Progress] at 1112.670 : Elaborate components

ERROR !
A null pointer access has been detected in the JVM.
This could happen when in your SpinalHDL description, you access an signal which is only defined further.
For instance :
  val result = Bool
  result := a ^ b  //a and b can't be accessed there because they are only defined one line below (Software rule of execution order)
  val a,b = Bool
          


: 

A fix could be:

In [22]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:02:59
[Progress] at 1137.803 : Elaborate components
[Progress] at 1137.804 : Checks and transforms
[Progress] at 1137.806 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1137.829
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire       [7:0]    a;

  assign a = 8'h2a;

endmodule



defined class TopLevel

**Issue explanation :**
SpinalHDL is not a language, it is a Scala library, which means it obeys the same rules as the Scala general purpose programming language. When you run your SpinalHDL hardware description to generate the corresponding VHDL/Verilog RTL, your SpinalHDL hardware description will be executed as a Scala programm, and a will be a null reference until the program executes val a = UInt(8 bits), so trying to assign it before then will result in a NullPointerException.

## Register defined as component input
In SpinalHDL, you are not allowed to define a component that has a register as an input. The reasoning behind this is to prevent surprises when the user tries to drive the inputs of child components with the registered signal. If a registered input is desired, you will need to declare the unregistered input in the io bundle, and register the signal in the body of the component.

In [23]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in(Reg(UInt(8 bits)))
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:05:26
[Progress] at 1284.515 : Elaborate components
[Progress] at 1284.517 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 1284.519 : Elaborate components
[Progress] at 1284.521 : Checks and transforms


: 

In [24]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:05:36
[Progress] at 1294.353 : Elaborate components
[Progress] at 1294.355 : Checks and transforms
[Progress] at 1294.357 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1294.384
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  input      [7:0]    io_a
);


endmodule



defined class TopLevel

If a registered a is really wanted, it can be done like so:

In [25]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
  val a = RegNext(io.a)
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:05:45
[Progress] at 1303.385 : Elaborate components
[Progress] at 1303.387 : Checks and transforms
[Progress] at 1303.390 : Generate Verilog
[Warning] 4 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1303.417
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  input      [7:0]    io_a,
  input               clk,
  input               reset
);
  reg        [7:0]    a;

  always @ (posedge clk) begin
    a <= io_a;
  end


endmodule



defined class TopLevel

## Scope violation
SpinalHDL will check that there are no signals assigned outside the scope they are defined in. This error isn’t easy to trigger as it requires some specific meta hardware description tricks.

In [26]:
class TopLevel extends Component {
  val cond = Bool()

  var tmp : UInt = null
  when(cond){
    tmp = UInt(8 bits)
  }
  tmp := U"x42"
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:07:57
[Progress] at 1435.730 : Elaborate components
[Progress] at 1435.732 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 1435.734 : Elaborate components
[Progress] at 1435.736 : Checks and transforms


: 

A fix could be:

In [27]:
class TopLevel extends Component {
  val cond = Bool()

  var tmp : UInt = UInt(8 bits)
  when(cond){

  }
  tmp := U"x42"
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:08:19
[Progress] at 1457.827 : Elaborate components
[Progress] at 1457.829 : Checks and transforms
[Progress] at 1457.832 : Generate Verilog
[Warning] 2 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1457.854
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire                cond;
  wire       [7:0]    tmp;

  assign tmp = 8'h42;

endmodule



defined class TopLevel

## Spinal can’t clone class  ???
This error happens when SpinalHDL wants to create a new datatype via the cloneOf function but isn’t able to do it. The reasons for this is nearly always because it can’t retreive the construction parameters of a Bundle.

In [28]:
//cloneOf(this) isn't able to retreive the width value that was used to construct itself
class RGB(width : Int) extends Bundle{
  val r,g,b = UInt(width bits)
}
class TopLevel extends Component {
  val tmp =  Stream(new RGB(8))  //Stream requires the capability to cloneOf(new RGB(8))
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:09:50
[Progress] at 1548.096 : Elaborate components
[Progress] at 1548.108 : Checks and transforms
[Progress] at 1548.111 : Generate Verilog
[Warning] 5 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1548.128
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire                tmp_valid;
  wire                tmp_ready;
  wire       [7:0]    tmp_payload_r;
  wire       [7:0]    tmp_payload_g;
  wire       [7:0]    tmp_payload_b;


endmodule



defined class RGB
defined class TopLevel

A fix could be:

In [29]:
case class RGB(width : Int) extends Bundle{
  val r,g,b = UInt(width bits)
}

class TopLevel extends Component {
  val tmp = Stream(RGB(8))
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:10:53
[Progress] at 1611.992 : Elaborate components
[Progress] at 1611.995 : Checks and transforms
[Progress] at 1611.997 : Generate Verilog
[Warning] 5 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1612.021
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire                tmp_valid;
  wire                tmp_ready;
  wire       [7:0]    tmp_payload_r;
  wire       [7:0]    tmp_payload_g;
  wire       [7:0]    tmp_payload_b;


endmodule



defined class RGB
defined class TopLevel

## Unassigned register
SpinalHDL will check that all registers which impact the design have been assigned somewhere.

In [30]:
class TopLevel extends Component {
  val result = out(UInt(8 bits))
  val a = Reg(UInt(8 bits))
  result := a
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:13:11
[Progress] at 1749.609 : Elaborate components
[Progress] at 1749.611 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 1749.614 : Elaborate components
[Progress] at 1749.615 : Checks and transforms


: 

A fix could be:

In [31]:
class TopLevel extends Component {
  val result = out(UInt(8 bits))
  val a = Reg(UInt(8 bits))
  a := 42
  result := a
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:13:18
[Progress] at 1756.463 : Elaborate components
[Progress] at 1756.465 : Checks and transforms
[Progress] at 1756.467 : Generate Verilog
[Done] at 1756.489
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  output     [7:0]    result,
  input               clk,
  input               reset
);
  reg        [7:0]    a;

  assign result = a;
  always @ (posedge clk) begin
    a <= 8'h2a;
  end


endmodule



defined class TopLevel

### Register with only init  
In some case, because of the design parameterization, it could make sense to generate a register which has no assignement but only a init statement.

In [32]:
class TopLevel(something: Boolean) extends Component {
  val result = out(UInt(8 bits))
  val a = Reg(UInt(8 bits))  
   
  if(something){ 
    a := 32
  }
  result := a
}
showRtl(new TopLevel(false))

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:13:37
[Progress] at 1775.407 : Elaborate components
[Progress] at 1775.409 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 1775.412 : Elaborate components
[Progress] at 1775.413 : Checks and transforms


: 

To fix it you can ask SpinalHDL to transform the register into a combinatorial one if no assignement is present but it has a init statement:

In [33]:
class TopLevel(something: Boolean) extends Component {
  val result = out(UInt(8 bits))
  val a = Reg(UInt(8 bits)).init(42).allowUnsetRegToAvoidLatch

  if(something){   
    a := 37   
  }
  result := a
}
showRtl(new TopLevel(false))

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:14:15
[Progress] at 1813.927 : Elaborate components
[Progress] at 1813.929 : Checks and transforms
[Progress] at 1813.933 : Generate Verilog
[Done] at 1813.954
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
  output     [7:0]    result,
  input               clk,
  input               reset
);
  wire       [7:0]    a;

  assign a = 8'h2a;
  assign result = a;

endmodule



defined class TopLevel

## Unreachable is statement
SpinalHDL will check all is statements in a switch are reachable.

In [36]:
class TopLevel extends Component {
  val sel = UInt(2 bits)
  val result = UInt(4 bits)
  switch(sel){
    is(0){ result := 4 }
    is(1){ result := 6 }
    is(2){ result := 8 }
    is(3){ result := 9 }
    is(0){ result := 2 } //Duplicated statement is statement !
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:15:24
[Progress] at 1882.732 : Elaborate components
[Progress] at 1882.738 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 1882.745 : Elaborate components
[Progress] at 1882.748 : Checks and transforms


: 

A fix could be:

In [37]:
class TopLevel extends Component {
  val sel = UInt(2 bits)
  val result = UInt(4 bits)
  switch(sel){
    is(0){ result := 4 }
    is(1){ result := 6 }
    is(2){ result := 8 }
    is(3){ result := 9 }
  }
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:15:39
[Progress] at 1897.522 : Elaborate components
[Progress] at 1897.526 : Checks and transforms
[Progress] at 1897.534 : Generate Verilog
[Warning] 2 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1897.564
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire       [1:0]    sel;
  reg        [3:0]    result;

  always @ (*) begin
    case(sel)
      2'b00 : begin
        result = 4'b0100;
      end
      2'b01 : begin
        result = 4'b0110;
      end
      2'b10 : begin
        result = 4'b1000;
      end
      default : begin
        result = 4'b1001;
      end
    endcase
  end


endmodule



defined class TopLevel

## Width mismatch
SpinalHDL will check that signals on the left and right side of assignments and operators are the same width.

In [38]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  val b = UInt(4 bits)
  b := a
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:16:10
[Progress] at 1928.574 : Elaborate components
[Progress] at 1928.575 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 1928.579 : Elaborate components
[Progress] at 1928.580 : Checks and transforms


: 

A fix could be:

In [39]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  val b = UInt(4 bits)
  b := a.resized
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:16:17
[Progress] at 1935.627 : Elaborate components
[Progress] at 1935.628 : Checks and transforms
[Progress] at 1935.631 : Generate Verilog
[Warning] 2 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1935.647
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire       [7:0]    a;
  wire       [3:0]    b;

  assign b = a[3:0];

endmodule



defined class TopLevel

###  Operator example
The following code:

In [40]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  val b = UInt(4 bits)
  val result = a | b
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:16:38
[Progress] at 1956.493 : Elaborate components
[Progress] at 1956.493 : Checks and transforms

**********************************************************************************************
[Warning] Elaboration failed (1 error).
          Spinal will restart with scala trace to help you to find the problem.
**********************************************************************************************

[Progress] at 1956.496 : Elaborate components
[Progress] at 1956.497 : Checks and transforms


: 

A fix could be:

In [41]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  val b = UInt(4 bits)
  val result = a | (b.resized)
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.23 10:16:47
[Progress] at 1965.814 : Elaborate components
[Progress] at 1965.815 : Checks and transforms
[Progress] at 1965.817 : Generate Verilog
[Warning] 3 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1965.839
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : 3c8f67ad383413c37b9e460fd52ce27764b102b4



module TopLevel (
);
  wire       [7:0]    _zz_1;
  wire       [7:0]    a;
  wire       [3:0]    b;
  wire       [7:0]    result;

  assign _zz_1 = {4'd0, b};
  assign result = (a | _zz_1);

endmodule



defined class TopLevel